Ejecutar: Lo que tiene un *

# Importamos Librerías (*)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from numpy import pi, sin, cos
from scipy.fft import fft, ifft
from scipy.io import wavfile
from scipy.signal import convolve,correlate,fftconvolve,lfilter
from scipy import signal
#from ipywidgets import interactive, interact
from matplotlib import rc
#import matplotlib.animation as animation
import librosa
rc('animation', html='jshtml')

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/MyDrive/Beca UNQ/Notebooks Python/')

In [ ]:
import os
pathCarriers = r"C:/Users/roman/OneDrive/Escritorio/Audio_Code/carriers/"
os.chdir(pathCarriers)

# Comparación de dos métodos

In [ ]:
sr = 48000
dur = 1
N = round(sr*dur)

Generamos unas sinusoides

In [ ]:
#Señal original
x = [.6*np.sin(2*np.pi*i/sr*200) + .4*np.sin(2*np.pi*i/sr*380 +0.2) for i in range(N)]
X = fft(x)
fs = [ sr/N*i for i in range(0, N//2) ] + [ sr/N*i for i in range(-N//2, 0) ]

fig, axs = plt.subplots(1,2, figsize = (12,5))

axs[0].set_title('Señal')
axs[0].plot(x[0:500])
axs[1].set_title('|DFT|')
axs[1].plot(fs, np.abs(X))
axs[1].set_xlim([-400,400])


display(Audio(x, rate =sr))

Hacemos 0 las frec negativas de la DFT, y tomamos IDFT para obtener la señal analítica derivada de la original.
La parte imaginaria es la transformada de Hilbert

In [ ]:
#Señal analitica
Z = np.zeros(N, dtype='complex')
for i in range(1,N//2): Z[i] = 2.*X[i]
z = ifft(Z)
H = np.imag(z)  #Transformada de Hilbert de x(t)

fig, axs = plt.subplots(1,4, figsize = (15,5))
axs[0].set_title('Señal Original')
axs[0].plot(x[0:500]) #Igual a x(t)
axs[1].set_title('Re(Señal Analítica)')
axs[1].plot(np.real(z[0:500])) #Igual a x(t)
axs[2].set_title('Im(Señal Analítica)')
axs[2].plot(H[0:500])
axs[3].set_title('|DFT(Señal Analítica)|')
axs[3].plot(fs, np.abs(Z))
axs[3].set_xlim([-400,400])

display(Audio(np.real(z), rate =sr))
display(Audio(H, rate =sr))

Obtenemos ahora la transformada de Hilbert a partir de una aproximación FIR

In [ ]:
M = 800 #Cantidad de puntos del FIR.
h = np.zeros(M)
for i in range(-M+1, M, 2): h[i] =  2/i/np.pi

H2 = np.convolve(x, h)[M-1:]

fig, axs = plt.subplots(3,3, figsize = (20,8))

axs[0][0].set_title('Señal original (x)')
axs[0][0].plot(x[2000:2500])
axs[1][0].set_title('Re(X(s)))')
axs[1][0].plot(fs, np.real(X))
axs[1][0].set_xlim([-400,400])
axs[2][0].set_title('Im(X(s))')
axs[2][0].plot(fs, np.imag(X))
axs[2][0].set_xlim([-400,400])


axs[0][1].set_title('HT -- IFFT (H)')
axs[0][1].plot(H[2000:2500])
axs[1][1].set_title('Re(DFT(H))')
axs[1][1].plot(fs, np.real(fft(H)))
axs[1][1].set_xlim([-400,400])
axs[2][1].set_title('Im(DFT(H))')
axs[2][1].plot(fs, np.imag(fft(H)))
axs[2][1].set_xlim([-400,400])

axs[0][2].set_title('HT -- FIR (H2)')
axs[0][2].plot(H2[2000:2500])
axs[1][2].set_title('Re(DFT(H2))')
axs[1][2].plot(fs, np.real(fft(H2)))
axs[1][2].set_xlim([-400,400])
axs[2][2].set_title('Im(DFT(H2))')
axs[2][2].plot(fs, np.imag(fft(H2)))
axs[2][2].set_xlim([-400,400])



display(Audio(H, rate =sr))
display(Audio(H2, rate =sr))

Voy entonces con el método FFT que parece ser el que mejor anda cuando hay buena resolución espectral

# **Encapsulo los métodos en una función (*):**

## Método principal (FFT)

In [ ]:
def analytic(x, circular=True):

  n = len(x) #Voy a apendear n zeros, así que duplico el tamaño
  if(not circular): n *= 2
  Z = np.zeros(n, dtype='complex')

  #Aplico una ventana para que la función periodizada sea continua - sino la envolvente exlplota si se extrae sobre muestras muy largas
  #x = np.array(x)* Tukey_window(len(x), 0.1)

  if(circular): X  = fft(x)
  else: X = fft( list(x) + len(x)*[0]) #acá appendeo los n 0's

  for i in range(1,n//2): Z[i] = 2.*X[i]
  Z[n//2] = X[n//2]
  z = ifft(Z)
  return z

def HT(x, circular = True):
  return np.imag(analytic(x, circular))

def lowpass_zerophase(x):
      b,a = signal.butter(2, 0.01, 'lp', output='ba')
      return signal.filtfilt(b,a, x)


In [ ]:
FILTER_ENVELOPE = False

In [ ]:
def graficarTodo(x, rango = 0, showFreq = True):
  if(rango==0): rango=len(x) #que cantidad de muestras del total grafico , [0:rango]
  a = analytic(x)
  env = np.abs(a)
  if(FILTER_ENVELOPE): env = lowpass_zerophase(env)

  if(showFreq):

    #Calculo frecuencia instantánea
    ph = np.angle(a)
    ph = np.unwrap(ph)
    #ph = signal.medfilt(ph, 3) #saco picos
    f = np.diff(ph)*sr/np.pi/2
    #f = signal.medfilt(f, 3) #saco picos

    #Descarto los valores basura
    #Si la amplitud es muy baja, la fase pierde su sentido. Hago un criterio muy simple: Si la amplitud promedio en un umbral es menor a tal valor, en ese ciclo dejo la frecuencia como "0"
    for i in range(0,len(f), 500):
      n = len(f[i:i+500])
      energy = np.sum(env[i:i+n])/n
      if(energy < 0.01): f[i:i+n] = np.zeros(n)


    #fig, axs = plt.subplots(3, figsize = (18,6))
    fig, axs = plt.subplots(2, figsize = (15,6))

    axs[0].set_title('Señal x(t) ;  Envolvente')
    axs[0].plot(x, color ='b', label= 'x(t)')
    axs[0].set_xlim([0,rango])

  # axs[1].set_title('Envolvente')
    axs[0].plot(env[:rango], 'r--', label='envelope') # color='r'
    axs[0].plot(-1*env[:rango], 'r--', label='envelope') # color='r'
    #axs[1].set_xlim([0,500])

    #axs[1].set_title('Fase')
    #axs[1].plot(ph[:rango])
    #axs[2].set_xlim([0,500])

    axs[1].set_title('Frecuencia')
    axs[1].plot(f[1:rango], color = 'orange', label = 'Frecuencia Instantánea')
    #axs[1].set_ylim([0,500])
    axs[1].plot(0)
    axs[1].set_xlim([0,rango])

  else:
    fig, axs = plt.subplots(1, figsize = (15,4))
    axs.set_title('Señal x(t) ;  Envolvente')
    axs.plot(x, color ='b', label= 'x(t)')
    axs.set_xlim([0,rango])

    axs.plot(env[:rango], 'r--', label='envelope') # color='r'
    axs.plot(-1*env[:rango], 'r--', label='envelope') # color='r'

  plt.show()

## Métodos alternativos (FIR)



---


Explicación: El método 1 proviene de samplear la rta  impulso (infinita) y hacer convolución.  Dejo una demo abajo, de como se puede obtener un aproximado a esa rta definiendo el espectro de esa IR con una FFT de N puntos y tomando ifft. Cuantos más puntos, más se parece a la respuesta del caso límite.


El método 2 es aplicar directamente la definición analógica a la discreta

Estudio abajo el método 1

Método 1:

In [ ]:
def HT_FIR(x):
  M = 1024 #Cantidad de puntos del FIR.
  h = np.zeros(M)
  #for i in range(-M+1,M): h[i] = 2/i/np.pi
  for i in range(-(M//2)+1, M//2+1, 2): h[i-M//2] =  2/i/np.pi  #respuesta a impulso discreta
  h *= Tukey_window(M, 0.1)

  H2 = np.convolve(x, h)
  return H2[M//2+1: len(x) + M//2+1]

def analyticFIR(x):
  return x + np.j*HT_FIR(x)


In [ ]:
def HT_FIR2(x):
  M = 1024//2 #Cantidad de puntos del FIR.
  h1 = [1/i/np.pi for i in range(-M,0)]
  h2 = [1/i/np.pi for i in range(1,M+1)]
  h = np.array(h1+h2)
  h *= Tukey_window(2*M, 0.1)

  H2 = np.convolve(x, h)
  return H2[M+1: len(x) + M+1]

def analyticFIR2(x):
  return x + np.j*HT_FIR(x)

In [ ]:
#Demostración: Tomo la respuesta a impulso a partir de la definición del espectro, y después calculo  lo mismo usando la fórmula cerrada
#Veo que ambas coinciden
N = 32
x = np.zeros(N, dtype='complex')
x[0]=0
for i in range(1,N//2): x[i]= -1j
for i in range(N//2+1,N): x[i]= 1j
h = ifft(x)
plt.plot(h)

h2 = np.zeros(N)
for i in range(1,N//2,2): h2[i] = 2/np.pi/i #términos positivos
for i in range(-N//2+1,0,2): h2[i] = 2/np.pi/(i) #términos negativos
plt.plot(h2)

err = np.array(h)-np.array(h2)
plt.plot(err)
errMax = max( (v, i) for i, v in enumerate(abs(err)) )
print('El máximo error relativo es ' + str(errMax[0]/ h[errMax[1]] ))

In [ ]:
fig, axs = plt.subplots(2)

H = fft(h)
axs[0].plot(np.real(H))
axs[0].plot(np.imag(H))

H2 = fft(h2)
axs[1].plot(np.real(H2))
axs[1].plot(np.imag(H2))

Y el método FFT es similar al Método FIR usando una respuesta a impulso del mismo largo que la señal de entrada y aplicando **circular convolution**. Esto se ve porque, si por ej, la señal de entrada y la IR tienen ambas 32 samples por ej, la DFT de la IR es exactamente -isgn(w), como se ve arriba

In [ ]:
wnoise = 2*(np.random.rand(N) - .5) #ruido del mismo largo de la señal
out = np.convolve(wnoise, h) #convolucion - ambas tienen la misma longitud, y diseñé h para que su fft sea justo la función signo que está arriba
#out tiene 2N-1 puntos

#Comparo contra el método fft:
out2 = HT(wnoise)  #Tiene N puntos

#ahora, la manera en que la fft hace el wrap-around es la siguiente:
out3 = np.array(out[:N-1])+ np.array(out[N:]) #suma de la primer mitad + segunda mitad invertida


#ahora sí: La convolución con la IR diseñada del mismo largo que la señal de entrada + sumarse a si misma invertida
# coincide con aplicar la FFT y

plt.plot(out2)
plt.plot(out3)

## Ventanas

In [ ]:
def Tukey_window(N, alpha):
  w = np.ones(N)
  for i in range(int(alpha*N//2)):
    v = .5 - .5*np.cos(2*pi*i/N/alpha)
    w[i] =  v
    w[N-i-1] = v
  return w

def Planck_window(N, eps):
  w = np.ones(N)
  w[0] = 0.
  for i in range(1,int(eps*N)):
    en = eps*N
    v = 1+ np.exp(en/i - en/(en-i))
    v= 1/v
    w[i] =  v
    w[N-i] = v
  return w

In [ ]:
#prueba de las ventanas
plt.figure(0)
plt.title('Planck Window')
plt.plot(Planck_window(1000,0.1))

plt.figure(1)
plt.title('Tukey Window')
plt.plot(Tukey_window(1000,0.1))

#Pruebas

In [ ]:
FILTER_ENVELOPE = True

In [ ]:
def compararMetodos(x, senialEsperada, envolvente =False, logScale = False, n =None, envRef = None):
  #n es la entrada
  fig, axs = plt.subplots(1,4, sharey= True)
  fig.set_size_inches(26, 8)
  for i in range(4):
     axs[i].set_ylim(-1.1,1.1)
     if(envolvente):
      axs[i].set_ylim(-.1,1.2)
     if(logScale): axs[i].set_yscale('log')

  if(envolvente): axs[0].set_title('Envolvente Esperada')
  else: axs[0].set_title('Señal Esperada')

  axs[0].plot(senialEsperada)

  for i, method, name in [(1,HT, "FFT"), (2, HT_FIR, "FIR 1"), (3, HT_FIR2, "FIR 2")]:

    out = method(x)
    if(envolvente):
      out = np.sqrt(x**2 + out**2)
      if(FILTER_ENVELOPE): out = lowpass_zerophase(out)


    axs[i].set_title(f'Método {name} \n Error RMS: {calcularErrorRMS(senialEsperada, out)}')
    axs[i].set_xlabel('Muestras [n]', fontsize = 15)
    if(n):
      axs[i].plot(n, out, label = 'Obtenida')
      axs[i].plot(n, senialEsperada, alpha = 0.4, color = 'red', label ='Esperada')
    else:
       axs[i].plot(out, label = 'Obtenida')
       axs[i].plot(senialEsperada, alpha = 0.4, color = 'red', label = 'Esperada')
    axs[i].legend()

def calcularErrorRMS(a,b, strng = True, recortarBordes = 0):
  i = recortarBordes
  f = len(a) - recortarBordes-1
  err = sum((np.array(a[i:f])-np.array(b[i:f]))**2)
  if(strng): return "{:0.2f}".format(err)
  else: return np.sqrt(err/len(a))

## Sobre Seniales Básicas

Verificación de que se cumplan las propiedades básicas

Verifico: La transformada de Hilbert de una constante es 0

In [ ]:
x=np.ones(5000)
esperada = np.zeros(5000)
compararMetodos(x, esperada)

Verifico: La transformada de Hilbert doble es "-" la original

In [ ]:
files = ['hspeech1 48khz.wav','hspeech2 48khz.wav', 'beatles-001.wav', 'cornoi.wav', 'popper1.wav', 'sample.wav']
s, sr = librosa.load(files[3])
compararMetodos(s,-s)

Verifico: La transformada de coseno es seno

In [ ]:
x = [np.cos(2*np.pi*i/sr*200)  for i in range(500)]
esperada =  [np.sin(2*np.pi*i/sr*200)  for i in range(500)]
compararMetodos(x, esperada)

Verifico: La transformada de 1/x^2+1 es x/x^2+1

In [ ]:
x = np.array([1/(i**2+1)  for i in range(-100,100)])
esperada =  [i/(i**2+1) for i in range(-100,100)]
compararMetodos(x, esperada)

Verifico: La transformada de sinc(x) es 1-cos(x)/x

In [ ]:
x = np.array([np.sin(i)/i  for i in range(-1001,1000,2)])
esperada =  [(1-np.cos(i))/i for i in range(-1001,1000,2)]
compararMetodos(x, esperada)

Verifico: La transformada de exp.cos es exp.sin

In [ ]:
sr = 44100
N = sr
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(N)])
env = np.array([np.exp(-i/10000) for i in range(N)])
s = np.array([np.sin(2*np.pi*i/sr*200) for i in range(N)])
compararMetodos(env*x, env*s, logScale= True)

## Sobre envolventes

Extracción de la envolvente exp.sin

In [ ]:
sr = 44100
N = sr
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(N)])
env = np.array([np.exp(-i/10000) for i in range(N)])
compararMetodos(env*x, env, envolvente = True)

Exponencial doble exp(-|t|)

In [ ]:
sr = 44100
N = sr
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(-N//2, N//2)])
env = np.array([np.exp(-abs(i)/10000) for i in range(-N//2, N//2)])
compararMetodos(env*x, env, envolvente = True)

Extracción de exp(-t^2).sin(t)

In [ ]:
sr = 44100
N = 2000
FILTER_ENVELOPE= False
x = np.array([np.cos(2*np.pi*i/sr*600) for i in range(-N//2, N//2)])
env = np.array([np.exp(-i*i/N/N*15) for i in range(-N//2, N//2)])
compararMetodos(env*x, env, envolvente = True)

Aplicando un pasabajos para suavizar la onda obtenida:

In [ ]:
plt.plot(x*env)

In [ ]:
sr = 44100
N = 2000

#t = np.linspace(0, sr/N, N)

FILTER_ENVELOPE= False
x = np.array([np.cos(2*np.pi*i/sr*600) for i in range(-N//2, N//2)]) + np.array([np.cos(2*np.pi*i/sr*700) for i in range(-N//2, N//2)])
env = np.array([np.exp(-i*i/N/N*15) for i in range(-N//2, N//2)])


fig, axs = plt.subplots(1,2, sharex= True)
fig.set_size_inches(13, 5)
axs[0].set_ylim(-1.1,1.1)
axs[1].set_ylim(-.1,1.2)

axs[0].set_title('Señal  $s[n] = e^{-n^2/100}.cos(2n\pi $ / 70)')
axs[1].set_title('Envolvente Esperada')

axs[0].set_xlabel('Muestras [n]')
axs[0].set_ylabel('Amplitud')
axs[1].set_xlabel('Muestras [n]')

axs[0].plot(x*env)
axs[1].plot(env)

plt.show()


In [ ]:
sr = 44100
N = 2000

#t = np.linspace(0, sr/N, N)

FILTER_ENVELOPE= False
x = np.array([np.cos(2*np.pi *i/sr*600 + (np.sin(2*np.pi*i/sr*500))) for i in range(-N//2, N//2)])
env = np.array([np.exp(-i*i/N/N*15) for i in range(-N//2, N//2)])


fig, axs = plt.subplots(1,2, sharex= True)
fig.set_size_inches(13, 5)
axs[0].set_ylim(-1.1,1.1)
axs[1].set_ylim(-.1,1.2)

axs[0].set_title('Señal  $s[n] = e^{-n^2/100}.cos(2n\pi $ / 70)')
axs[1].set_title('Envolvente Esperada')

axs[0].set_xlabel('Muestras [n]')
axs[0].set_ylabel('Amplitud')
axs[1].set_xlabel('Muestras [n]')

axs[0].plot(x*env)
axs[1].plot(env)

plt.show()

In [ ]:
compararMetodos(env*x, env, envolvente = True)

In [ ]:
sr = 44100
N = 2000

#t = np.linspace(0, sr/N, N)

FILTER_ENVELOPE= False

dom = range(-N//2, N//2)

x = np.array([np.cos(2*np.pi*i/sr*600) for i in dom ])
env = np.array([np.exp(-i*i/N/N*15) for i in dom ])


fig, axs = plt.subplots(1,2, sharex= True)
fig.set_size_inches(13, 5)
axs[0].set_ylim(-1.1,1.1)
axs[1].set_ylim(-.1,1.2)

axs[0].set_title('Señal de entrada \n $s[n] = e^{-n^2/100}.cos(2n\pi $ / 70)', fontsize = 15)
axs[1].set_title('Envolvente Esperada \n  env[n] = $e^{-n^2/100}$', fontsize = 15)

axs[0].set_xlabel('Muestras [n]', fontsize = 14)
axs[0].set_ylabel('Amplitud', fontsize = 14)
axs[1].set_xlabel('Muestras [n]', fontsize = 14)

axs[0].plot(dom, x*env)
axs[1].plot(dom, env)

plt.show()


In [ ]:
compararMetodos(env*x, env, envolvente = True, n = dom)

In [ ]:
FILTER_ENVELOPE = True
compararMetodos(env*x, env, envolvente = True, n = dom)

In [ ]:
H =HT_FIR2(x*env)
E = np.sqrt(H**2 + (x*env)**2)
plt.plot(E)

In [ ]:
sos = signal.butter(2, 0.01, 'lp', output='sos')
filtered = signal.sosfilt(sos, E)
plt.plot(filtered)
(max(filtered))

Y filtradas

In [ ]:
sr = 44100
N = 2000
FILTER_ENVELOPE= True
x = np.array([np.cos(2*np.pi*i/sr*600) for i in range(-N//2, N//2)])
env = np.array([np.exp(-i*i/N/N*15) for i in range(-N//2, N//2)])
compararMetodos(env*x, env, envolvente = True)

extraccion de rampa lineal

In [ ]:
sr = 44100
N = sr
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(-N//2, N//2)])
env = np.array([abs(i)/N for i in range(-N//2, N//2)])
compararMetodos(env*x, env, envolvente = True)

In [ ]:
sr = 44100
N = sr
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(-N//2, N//2)])
env = np.array([i/N for i in range(N//2)]+ [i/N for i in range(N//2,0,-1)])
compararMetodos(env*x, env, envolvente = True)

extraccion de parábola

In [ ]:
sr = 44100
N = sr
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(-N//2, N//2)])
env = np.array([8*i*i/N/N for i in range(N//2)]+ [8*i*i/N/N for i in range(N//2,0,-1)])
compararMetodos(env*x, env, envolvente = True)

## Sobre transientes

In [ ]:
sr = 44100
N = sr*2
zer = np.zeros(N)
x = zer
env = zer
x[20000:40000] += np.array([np.cos(2*np.pi*i/sr*200) for i in range(20000)])
env[20000:40000] = np.array([np.exp(-abs(i)/10000) for i in range(20000)])

compararMetodos(env*x, env, envolvente = True)

In [ ]:
sr = 44100
N = sr
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(-N//2, N//2)])
env = np.array([3*i*i/N/N for i in range(N//2)]+ [i*i/N/N for i in range(N//2,0,-1)])
compararMetodos(env*x, env, envolvente = True)

Sin filtrar

In [ ]:
sr = 44100
N = sr
FILTER_ENVELOPE = False
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(-N//2, N//2)])
env = np.array([i/N for i in range(N//2)]+ [2*i/N for i in range(N//2,0,-1)])
compararMetodos(env*x, env, envolvente = True)

Filtrando

In [ ]:
sr = 44100
N = sr
FILTER_ENVELOPE = True
x = np.array([np.cos(2*np.pi*i/sr*200) for i in range(-N//2, N//2)])
env = np.array([i/N for i in range(N//2)]+ [2*i/N for i in range(N//2,0,-1)])
compararMetodos(env*x, env, envolvente = True)

In [ ]:
sr = 44100
N = 2000
FILTER_ENVELOPE = True
dom = range(-N//2, N//2)
x = np.array([np.cos(2*np.pi*i/sr*200) for i in dom])
env = np.array([i/N for i in range(N//2)]+ [2*i/N for i in range(N//2,0,-1)])


plt.plot(dom, env*x)
plt.plot(dom,env)

In [ ]:
FILTER_ENVELOPE = True
compararMetodos(env*x, env, envolvente = True)

In [ ]:
sr = 44100
N = 2000
FILTER_ENVELOPE = True
dom = range(-N//2, N//2)
x = np.array([np.cos(2*np.pi*i/sr*200) for i in dom])
env = np.array([i for i in range(950)]+ [i +(i-950)*i/30 for i in range(950,1000)] + [i +50*1000/30 +(i-1000)*i/20 for i in range(1000,1050)] + [1050+50*1000/30+50*1050/20-3*(i-1050) for i in range(1050,2000)])/N/2.5

fig, ax = plt.subplots(1, figsize = (6,8))

plt.plot(dom, env*x, label ='s[n]')
plt.plot(dom,env, label = 'env[n]')
plt.title('$s[n]$ = cos($2\pi.n/220$). $env[n]$', fontsize = 16)
plt.xlabel('Muestras [n]', fontsize=13)
plt.ylabel('Amplitud', fontsize = 13)
plt.legend()


In [ ]:
FILTER_ENVELOPE = False
compararMetodos(env*x, env, envolvente = True, n = dom)

In [ ]:
FILTER_ENVELOPE = True
compararMetodos(env*x, env, envolvente = True, n = dom)

Pruebas básicas sobre el método FFT

In [ ]:
x = [np.exp(-i/1000)*np.sin(2*np.pi*i/sr*200) for i in range(N)]
graficarTodo(x, rango = 5000)

In [ ]:
files = ['hspeech1 48khz.wav','hspeech2 48khz.wav', 'beatles-001.wav', 'cornoi.wav', 'popper1.wav', 'sample.wav']
s, sr = librosa.load(files[3])
x = HT(HT(s))
display(Audio(x, rate = sr))
plt.plot(s[4000:5000])
plt.plot(x[4000:5000])
plt.plot((np.array(s)+np.array(x))[4000:5000]) #si la suma se cancela, es que está bien!

In [ ]:
#Verifico: La transformada de Hilbert de cos es sen:
x = [np.cos(2*np.pi*i/sr*200)  for i in range(500)]
H = HT(x)
env = [np.sqrt(x[i]**2+H[i]**2)  for i in range(len(x))]
plt.plot(x)
plt.plot(H)
plt.plot(env)

# Análisis de señales sintetizadas (método FFT)
Analizo frecuencia y amplitud de algunas señales

In [ ]:
sr = 48000
dur = 1
N = round(sr*dur)

In [ ]:
x = [np.sin(2*np.pi*i/sr*200) + np.sin(2*np.pi*i/sr*380 +0.2) for i in range(N)]
graficarTodo(x, range = 1000)

In [ ]:
x = [np.sin(2*np.pi*i/sr*200) for i in range(N)]
graficarTodo(x, range = 5000)

In [ ]:
x = [np.exp(-i/1000)*np.sin(2*np.pi*i/sr*200) for i in range(N)]
graficarTodo(x, range = 5000)

In [ ]:
x = [np.exp(-np.abs((i-5000)/1000))*np.sin(2*np.pi*i/sr*200) for i in range(N)]
graficarTodo(x, range = 10000)

In [ ]:
graficarTodo(x, range = 20000)

Vemos que la información de la frecuencia y fase hay que ignorarla una vez que la amplitud de la señal es muy chica, en un intervalo grande, pero también en un sampl.
Esto nos evita tener que descartar saltos abruptos en la señal de frecuencia instantánea, que nacen de saltos grandes en la señal de fase instantánea)

In [ ]:
x = [np.sin(2*np.pi*i/sr*200)*np.cos(2*np.pi*i/sr*10) for i in range(N)]
graficarTodo(x, range=8000)

In [ ]:
x = [np.sin(2*np.pi*i/sr*200)+np.cos(2*np.pi*i/sr*10) for i in range(N)]
graficarTodo(x, range=8000)

In [ ]:
x = [np.sin(2*np.pi*i/sr*200)+np.cos(2*np.pi*i/sr*100) for i in range(N)]
graficarTodo(x, range=5000)

In [ ]:
x = [np.sin(2*np.pi*i/sr*200)*np.cos(2*np.pi*i/sr*50) for i in range(N)]
graficarTodo(x, range=8000)

In [ ]:
x = [np.sin(2*np.pi*i/sr*250)*(1-i*i/N/N) for i in range(N)]
graficarTodo(x, range = N)

In [ ]:
from ipywidgets import interact

def f(freq):
  x = [np.exp(-i/4000)*np.sin(2*np.pi*i/sr*200) + np.exp(-i/2500)*np.sin(2*np.pi*i/sr*freq) for i in range(2*N)]
  graficarTodo(x, range = 7000)
  display(Audio(x, rate = sr))

  #Envolvente de la envolvente:
  '''
  env = np.abs(analytic(x))
  env = np.abs(analytic(env**2)) #Envolvente de la norma al cuadrado
  env = np.sqrt(env)
  plt.plot(x[:2000])
  plt.plot(env[0:2000])
  '''

  return

interact(f, freq=300)

#Tiende a calcular la frecuencia promedio como la frecuencia, y la sinusoide de frecuencia diferencial como envolvente.
#O sea, a usar la fórmula sen(f1) + sen(f2) = sen(f1+f2 /2).cos(f1-f2 /2)

Si la función no es suave, la envolvente se vuelve loca: pruebas con |sin(t)|

In [ ]:
x = [np.abs(np.sin(2*np.pi*i/sr*150)) for i in range(N)]
graficarTodo(x, range = 10000)
Audio(x, rate = sr)

|sin(t)|.exp(-t)

In [ ]:
x = [np.abs(np.sin(2*np.pi*i/sr*250))*np.exp(-i/10000) for i in range(N)]
graficarTodo(x, range = 6000)

|sin(t)|.exp(-t)(1-t^2)

In [ ]:
x = [np.abs(np.sin(2*np.pi*i/sr*250))*(1-i/N)*np.exp(-i/10000) for i in range(N)]
graficarTodo(x, range = 10000)

Probando con dos pasadas: Tomamos la envolvente de la envolvente y vemos si eso resulta en algo

In [ ]:
from ipywidgets import interact

def f(freq):
  x = [np.sin(2*np.pi*i/sr*200)*np.cos(2*np.pi*i/sr*freq) for i in range(N)]
  graficarTodo(x, range = 10000)
  display(Audio(x, rate = sr))
  env = np.abs(analytic(x))
  env = env**2 #norma al cuadrado, para ser diferenciable y que no se vuelva loca
  env = np.abs(analytic(env))
  plt.figure(figsize = (15,6))
  plt.plot(np.sqrt(2*env)) #Tomo raíz cuadrada, no porque sea correcto, sino para compensar un poco el haber elevado al cuadrado antes. La constante la puse a mano después de ver el gráfico
  plt.plot(x)
  return

interact(f, freq=10)

Y, resulta bárbaro, pero no tira la amplitud real, sino una escalada por una constante.
Hacemos otra prueba con la suma de tres sinusoides en relación armónica:

  ## Análisis de señales transientes

Envolvente filtrada

In [ ]:
sr = 44100
N = round(sr*0.1)
fase1 = [2*pi*800*i/sr +0.2 + 0.2*cos(i*0.2) for i in range(N)]
fase2 = [2*pi*930*i/sr +0.4 + 0.1*sin(i*0.12) for i in range(N)]

s1 = cos(fase1) *.5
s2 = cos(fase2) *.5

env = [0 for i in range(N//4)] + [np.exp(-i*6/N) for i in range(N//2)] + [np.exp(-i*2/N) for i in range(N//4+1)]
s= np.array(env)*(s1+s2)
FILTER_ENVELOPE = True
graficarTodo(s, showFreq=False)

Sin filtrar

In [ ]:
#Sin filtrar:
FILTER_ENVELOPE = False
graficarTodo(s, showFreq=False)

# Análisis de señales sintetizadas periódicas

In [ ]:
sr = 48000
dur = 1
N = round(sr*dur)

def sinSig(freq, Amp=0.5, phase=0, damp=5, len = N):
  x = [Amp*np.sin(2*np.pi*i/sr*freq+phase)*np.exp(-i*damp/sr) for i in range(N)]
  return np.array(x)

def buzz(freq, Amp):
  x = np.ones(N)
  for i in range(N):
     if(i % (sr/freq) == 0): x[i]=-1.
  return np.array(x)

def sq(freq, Amp):
  x = [1. for i in range(sr//freq)] #primer ciclo
  for i in range(round(sr/freq/2)):
    x[i] = -1. #primer medio ciclo
  x *= round(freq/sr*N) #periodizo hasta la cantidad de samples necesaria
  while(len(x)<N): x += [0]
  return np.array(x)

In [ ]:
x = sinSig(400, 0.5, 0, 7) + sinSig(800, 0.3, 0, 9)
graficarTodo(x, range = 5000)
Audio(x, rate = sr)

In [ ]:
x = buzz(400, 0.2)
graficarTodo(x, range = 200)
Audio(x, rate = sr)

In [ ]:
x = sq(400, 0.2)
graficarTodo(x, range = 2000)
Audio(x, rate = sr)

In [ ]:
x = sq(400, 0.2) + sq(600, 0.1)
graficarTodo(x, range = 2000)
Audio(x, rate = sr)

In [ ]:
x = sq(400, 0.4) + sq(600, 0.2) + sq(800, 0.1)
graficarTodo(x, range = 2000)
Audio(x, rate = sr)

In [ ]:
x = sinSig(400, Amp=0.4, damp = 0) + sinSig(600, Amp=0.3, damp=0) + sinSig(800, Amp=0.2, damp=0)
graficarTodo(x, range = 2000)
display(Audio(x, rate = sr))

#Envolvente de la envolvente:
print('Envolvente de la envolvente')
env = np.abs(analytic(x)) #Envolvente
env = np.abs(analytic(env**2)) #Envolvente del cuadrado de la envolvente
env = np.sqrt(env) #Raiz para compensar
plt.figure(figsize = (17,3))
plt.plot(x[:2000])
plt.plot(env[0:2000])

In [ ]:
plt.plot(x[:2000])
plt.plot(env[0:2000])

#Envolvente de la envolvente de la envolvente:
print('Envolvente de la envolvente de la envolvente:')
env = np.abs(analytic(env**2)) #Envolvente de la norma al cuadrado
env = np.sqrt(env)*2

plt.plot(env[0:2000])

In [ ]:
x = sinSig(400, Amp=0.4, damp = 4) + sinSig(600, Amp=0.3, damp=6) + sinSig(800, Amp=0.2, damp=8)
graficarTodo(x, range = 2000)
display(Audio(x, rate = sr))

#Envolvente de la envolvente:
env = np.abs(analytic(x))
env = np.abs(analytic(env**2)) #Envolvente de la norma al cuadrado
env = np.sqrt(env)
plt.plot(x[:2000])
plt.plot(env[0:2000])

In [ ]:
#Envolvente de la envolvente de la envolvente (ya no mejora...)
env = np.abs(analytic(env**2)) #Envolvente de la norma al cuadrado
env = np.sqrt(env)
plt.plot(x[:2000])
plt.plot(env[0:2000])

Ruido filtrado con pasabanda butter:

In [ ]:
from ipywidgets import interact

def f(fc, bw):
  orden = 6
  fizq = fc -bw/2
  fder = fc +bw/2
  wnoise = 2*(np.random.rand(N) - .5) #ruido del mismo largo de la señal
  sos = signal.butter(orden/2, [fizq,fder], 'bandpass', fs=sr, output='sos') #divido por 2 porque la implementación pide la cantidad de secciones de orden 2 para filtros BP
  x = signal.sosfilt(sos, wnoise) * 10

  graficarTodo(x)
  display(Audio(x, rate = sr))
  return

interact(f, fc=(200, 400), bw = (20, 100))


Esto anda bastante bien! Funciona mucho mejor de lo esperado, tanto para la envolvente, como para la frecuencia (que debe ser promediada)
Si, de vuelta, pedimos la envolvente de la envolvente:

In [ ]:
#Envolvente de la envolvente:
env = np.abs(analytic(x))**2
env = np.abs(analytic(env))
env = np.sqrt(env)
plt.plot(env)
plt.plot(x)

#Definición del banco de filtros (*)

In [ ]:
def filter_bank(s, freqs, fs=sr, orden = 6, bw = 1): #bw va entre 0 y 1. Hace más angostas las bandas centraadas en las frecuencias "freqs"
  N = len(freqs)

  #Out (array con todas las señales filtradas)
  filtered_bank = []

  fizq = 150 # freq izq de la primer banda, la asumo

  #e1 --> N-1
  for i in range(N-1):


    fc = freqs[i]
    fder = freqs[i+1]

    f1 = fc - (fc-fizq)/2*bw
    f2 = fc + (fder-fc)/2*bw
    fizq = fc

    sos = signal.butter(orden/2, [f1,f2], 'bandpass', fs=fs, output='sos') #divido por 2 porque la implementación pide la cantidad de secciones de orden 2 para filtros BP
    filtered_bank.append(signal.sosfilt(sos, s))

  #eN --> Nyquist

  #Pasaaltos
  sos = signal.butter(orden, freqs[-1], 'highpass', fs=fs, output='sos')
  filtered_bank.append(signal.sosfilt(sos, s))

  return filtered_bank

In [ ]:
freqs_AB = [333, 455, 540, 642, 762, 906, 1076, 1278, 1518, 1803, 2142, 2544, 3022, 3590, 4264, 6645]
freqs_MED = [120,235,384,579,836, 1175,1624,2222,3019,4084,5507,7410]

# Análisis de señales reales

Analizo señales melódicas, primero como vienen, y luego filtradas a través de un banco de filtros pasabanda.
Sacar la envolvente de la señal completa, antes de pasarla por el banco de filtros, no tiene ninguna utilidad ni importancia. Dejo algunas a modo de curiosidad simplemente.

## Melodía localmente estacionaria (corno inglés)

In [ ]:
s, sr = librosa.load('cornoi.wav') #Fundamental comprendida entre 250hz y 415hz
x = s[:sr*6]
graficarTodo(x)
display(Audio(x, rate = sr))

Analizamos más de cerca esta forma de onda...

In [ ]:
from ipywidgets import interact

def g(i,n):
  x = s[round(sr*i):round(sr*i + n)]
  plt.plot(x)
  xperiodic = []
  while(len(xperiodic) < sr): xperiodic += list(x) #append
  display(Audio(xperiodic, rate = sr))
  return

interact(g, i = (0, len(x)/sr), n=(0, sr))

Pasándola por el banco de filtros:

In [ ]:
from ipywidgets import interact

def g(i,n, fi):
  filtradas = filter_bank(s, freqs_AB, sr, orden = 10, bw = 0.5)
  x= filtradas[fi]
  print('Frecuencia Central del filtro: ' + str(freqs_AB[fi]))
  graficarTodo(x[i:i+n], showFreq=True)
  display(Audio(x, rate = sr))

interact(g, i = (0, len(x)), n = (0, sr), fi = (0, len(freqs_AB)))

Comentarios:

1) Al usar una banda angosta, la señal resultante termina pareciéndose a una sinusoide y la envolvente es dentro de todo fiel


2) Una melodía simple como esta (Db3 - Ab3) tiene su fundamental siempre adentro de la primer banda de análisis, con lo cual, a falta de estructura fina, con la envolvente sola de esa banda no hago NADA.



##Melodía localmente percusiva (cello stacc)

In [ ]:
s, sr = librosa.load('drive/MyDrive/Audios/' + 'popper1.wav')
x = s[:sr*6]
graficarTodo(x, showFreq=False)
display(Audio(x, rate = sr))

In [ ]:
from ipywidgets import interact

def g(ti,n):
  x = s[round(sr*ti):round(sr*ti + n)]
  graficarTodo(x, showFreq = False)
  display(Audio(x, rate = sr))
  return

interact(g, ti = (0, len(s)/sr), n=(0, sr))

## Melodía localmente percusiva (clave)



In [ ]:
s, sr = librosa.load('drive/MyDrive/Audios/' + 'bach8.wav')
x = s[:sr*6]
graficarTodo(x, showFreq = False)
display(Audio(x, rate = sr))

In [ ]:
from ipywidgets import interact

def g(ti,n):
  x = s[round(sr*ti):round(sr*ti + n)]
  graficarTodo(x, showFreq=False)
  display(Audio(x, rate = sr))
  return

interact(g, ti = (0, len(s)/sr), n=(0, sr))

## Oboe Staccato

In [ ]:
s, sr = librosa.load('obStacc.wav')
x = s[sr*8:sr*9]
graficarTodo(x, showFreq = False)
display(Audio(x, rate = sr))

In [ ]:
from ipywidgets import interact

def g(i,n, fi):
  filtradas = filter_bank(s, freqs_AB, sr, orden = 10, bw = 0.5)
  x= filtradas[fi]
  print('Frecuencia Central del filtro: ' + str(freqs_AB[fi]))
  graficarTodo(x[i:i+n], showFreq=True)
  display(Audio(x, rate = sr))

interact(g, i = (0, len(x)), n = (0, sr), fi = (0, len(freqs_AB)))

# Conclusiones

El método es excelente cuando se trata de una ÚNICA sinusoide dampeada o modulada en amplitud muy lentamente o por una función monótona y suave. Es decir, cuando se cumplen las condiciones del teorema de Bedrossian: f(t)= s(t)*env(t), con s(t) una señal cuya F.T. tiene todas frecuencias mayores que las de env(t)

En el caso particular de que s(t) sea una sinusoide y e(t) cualquier señal de variación lenta, digamos, menor a 200hz o 100hz, anda muy bien la extracción de envolvente e incluso de frecuencia (que en este caso no nos interesa obtener).

En el caso de la salida de un filtro pasabandas, de vuelta, funciona muy bien si el filtro es angosto o si justo se puede descomponer en "una unica sinusoide modulada".

El problema está en la suma de dos sinusoides: Por la identidad de la suma = dif*sum etc, se va a tomar como envolvente a una señal que, para nosotros, puede ser también de alta frecuencia. En la matemática anda bien, pero en lo que queremos lograr no, no es lo que queremos.
(se toma como "envolvente" a la señal lenta en la identidad sin(f1t)+sin(f2t)= cos(f1-f2 /2 t)sen(f1+f2 /2 t) )


Algo similar puede ocurrir en la salida del pasabandas según la señal de entrada o si la banda no es muy angosta. Ya se pierde un poco la interpretación de "envolvente" y ni siquiera se puede obtener la frecuencia instantánea (o frecuencia real, si la función que resulta es periódica) en la generalidad de los casos. A tener en cuenta a la hora de estudiar donde falla esta extracción.


-Si la señal tiene picos o es muy discontinua, la extracción de envolvente anda mal en las transientes, pero mejora bastante con un filtro aplicado a la envolvente.

-Si se trata de suma de dos sinusoides, extraer la envolvente de la envolvente anda bien en algunos casos.
No exploro y dejo de lado ese camino.

-Si se trata de ruido blanco filtrado por un pasabanda, también anda muy bien.
Con señales musicales, en la parte estacionaria anda muy bien. En la transiente también, mejor de lo esperado, para instrumentos que no sean de percusión (no estudié estos últimos).

# Comparación con la librería scipy

In [ ]:
import scipy

In [ ]:
x = np.exp([-np.abs(i)/500 for i in range(-1000,1000)])* np.sin([i*2*pi*200/44100 for i in range(2000)])
smia = HT(x)
sotra = scipy.signal.hilbert(x)
#plt.plot(x)
plt.plot(np.sqrt(smia**2 + x**2))
plt.plot(np.abs(sotra))
plt.plot(smia-np.imag(sotra))

In [ ]:
print(max(smia-np.imag(sotra)))